In [1]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
import os
import random
from PIL import Image
import copy
import umap
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics import normalized_mutual_info_score

In [25]:
# cargo listas de train, val y test
import pickle
with open('data/test', 'rb') as file:
    test = pickle.load(file)
with open('data/test_labels', 'rb') as file:
    test_labels = pickle.load(file)
with open('data/validation', 'rb') as file:
    validation = pickle.load(file)
with open('data/validation_labels', 'rb') as file:
    validation_labels = pickle.load(file)
with open('data/train', 'rb') as file:
    train = pickle.load(file)
with open('data/train_labels', 'rb') as file:
    train_labels = pickle.load(file)

In [26]:
scaled_train = StandardScaler().fit_transform(train)
scaled_val = StandardScaler().fit_transform(validation)
scaled_test = StandardScaler().fit_transform(test)

In [27]:
reducer300 = umap.UMAP(random_state=22, n_components=300, n_neighbors=45)
reducer300.fit(scaled_train)

UMAP(n_components=300, n_neighbors=45, random_state=22, tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True})

In [28]:
valReduced300 = reducer300.transform(scaled_val)

/mnt/882C57D72C57BEBE/Anaconda3/lib/python3.8/site-packages/scipy/sparse/_index.py:124: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [29]:
testReduced300 = reducer300.transform(scaled_test)

In [31]:
# los guardo porque tardaron un buen rato
with open('data/trainReduced300', 'wb') as file:
    pickle.dump(reducer300.embedding_, file)
with open('data/valReduced300', 'wb') as file:
    pickle.dump(valReduced300, file)
with open('data/testReduced300', 'wb') as file:
    pickle.dump(testReduced300, file)

In [ ]:
with open('data/trainReduced300', 'rb') as file:
    trainReduced300 = pickle.load(file)
with open('data/valReduced300', 'rb') as file:
    valReduced300 = pickle.load(file)
with open('data/testReduced300', 'rb') as file:
    testReduced300 = pickle.load(file)

In [9]:
kmeans = KMeans(n_clusters=28, n_init=5, verbose=10, random_state=22)
kmeans.fit(reducer300.embedding_) #si no existe reducer300 es porque hay que cambiar reducer300.embedding_  por trainReduced300

Initialization complete
Iteration 0, inertia 59385.8984375
Iteration 1, inertia 53306.921875
Iteration 2, inertia 53249.74609375
Iteration 3, inertia 53247.48046875
Iteration 4, inertia 53245.37890625
Iteration 5, inertia 53237.99609375
Iteration 6, inertia 53190.96484375
Iteration 7, inertia 53077.5
Iteration 8, inertia 52977.25
Iteration 9, inertia 52969.45703125
Converged at iteration 9: strict convergence.
Initialization complete
Iteration 0, inertia 56633.29296875
Iteration 1, inertia 51482.1484375
Iteration 2, inertia 51456.48046875
Iteration 3, inertia 51456.3984375
Converged at iteration 3: center shift 3.3054686809919076e-06 within tolerance 4.663771092891693e-05.
Initialization complete
Iteration 0, inertia 59146.7265625
Iteration 1, inertia 54215.43359375
Iteration 2, inertia 54205.80078125
Iteration 3, inertia 54205.72265625
Converged at iteration 3: strict convergence.
Initialization complete
Iteration 0, inertia 61336.4921875
Iteration 1, inertia 52632.4765625
Iteration 2

KMeans(n_clusters=28, n_init=5, random_state=22, verbose=10)

In [10]:
# descargo modelo
with open('models/kmeansBaseline', 'wb') as file:
    pickle.dump(kmeans, file)

In [32]:
# cargo modelo
with open('models/kmeansBaseline', 'rb') as file:
    kmeans = pickle.load(file)

In [33]:
kmeans.n_clusters

28

In [11]:
kmeans.labels_

array([ 1,  1, 26, ..., 12,  3, 26], dtype=int32)

In [12]:
import numpy as np
from sklearn import metrics
# https://nlp.stanford.edu/IR-book/html/htmledition/evaluation-of-clustering-1.html#fig:clustfg3
# https://stackoverflow.com/questions/34047540/python-clustering-purity-metric
def purity_score(y_true, y_pred):
    # compute contingency matrix (also called confusion matrix)
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    # return purity
    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix)

In [14]:
purity_score(train_labels, kmeans.labels_)

0.6342834203131272

In [15]:
normalized_mutual_info_score(train_labels, kmeans.labels_)

0.7012983135535616

In [17]:
# validation score
val_preds = kmeans.predict(valReduced300)

In [18]:
purity_score(validation_labels, val_preds)

0.632368703108253

In [19]:
normalized_mutual_info_score(validation_labels, val_preds)

0.7058476819209059

In [21]:
# test score
test_preds = kmeans.predict(testReduced300)

In [22]:
purity_score(test_labels, test_preds)

0.6420150053590568

In [23]:
normalized_mutual_info_score(test_labels, test_preds)

0.7005482788789146